As you can see from the examples in simple_service_discovery,
a search will often return more than one record,
and so sometimes you need to review some of the
resource metadata to determine which one or ones you want.  You may
have noticed that the results behave similarly to the results from the
data access services. Like them, registry search results are returned
as a `~pyvo.registry.regtap.RegistryResults` instance, and each
record is represented as a `~pyvo.registry.regtap.RegistryResource` instance. 

In [ ]:
import pyvo as vo
colls = vo.regsearch(keywords=["NVSS"], servicetype='sia')
nvss = colls[0]
nvss.res_title

If you are looking for a particular data collection or catalog, as we
did above when we looked for the NVSS archive, often simply reviewing
the titles is sufficient.  Other times, particularly when you are not
sure what you are looking for, it helps to look deeper.  

The resource description, available via the 
`~pyvo.registry.regtap.ResourceRecord.res_description` property,
tends to be the most revealing.  It contains a paragraph (or two)
summarizing the catalog or data collection.  It will often describe
the scientific intent behind the collection.  

The `~pyvo.registry.regtap.RegistryResource.short_name` can also be
helpful, as well.  This name is meant to be short--16 characters or
fewer; consequently, the value is often includes the abbreviation for the
project or observatory that produced the collection or catalog.  

A selection of the resource metadata, including the title, shortname and
desription, can be printed out in a summary form with
the `~pyvo.registry.regtap.RegistryResource.describe` function.

In [ ]:
nvss.describe()

Registry Records can be converted to a service object that can be queried.  

In [ ]:
nvss = colls[0].service  # converts record to service object
nvss.search(pos=(350.85, 58.815),size=0.25,format="image/fits").to_table()

Thus, not only does this service instance contain the base URL but it
also includes all of the metadata from the registry that desribes the
service.  With this service object, we can either call its 
`~pyvo.dal.sia.SIAService.search` function directly as above, or 
create query objects to get cutouts for a whole list of sources.

In [ ]:
cutouts1 = nvss.search(pos=(148.8888, 69.065), size=0.2)
nvssq = nvss.create_query(size=0.2)  # or create a query object
nvssq.pos = (350.85, 58.815)
cutouts2 = nvssq.execute()

Our discussion of service metadata offers an opportunity to highlight
another important property, the service's *IVOA Identifier* (sometimes
referred to as its *ivoid*).  This is a globally-unique identifier
that takes the form of a 
`URI <http://en.wikipedia.org/wiki/Uniform_resource_identifier>`_:

In [ ]:
colls = vo.regsearch(keywords=["NVSS"], servicetype='sia')
for coll in colls:
    print(coll.ivoid)

This identifier can be used to retrieve a specific service from the registry.  

In [ ]:
nvss = vo.registry.ivoid2service('ivo://nasa.heasarc/skyview/nvss',servicetype='sia')
nvss.search(pos=(350.85, 58.815),size=0.25,format="image/fits").to_table()

As we end this discussion of the service objects, you can hopefully
see that there is a straight-forward chain of discovery classes that
connect the registry down through to a dataset.  Spelled out in all
its detail, it looks like this:

In [ ]:
services = vo.regsearch(keywords=["NVSS"],
                        servicetype='sia') 
nvss_resource = services[0]
nvss_service = nvss_resource.service 
nvss_query = nvss_service.create_query(pos=(350.85, 58.815),
                      size=0.25, 
                      format="image/fits")
images = nvss_query.execute()
firstim = images[0]

print(type(services))
print(type(nvss_resource))
print(type(nvss_service))
print(type(nvss_query))
print(type(images))
print(type(firstim))

Most of the time, it's not necessary to follow all these steps
yourself, so there are functions and methods that provide syntactic
shortcuts.  However, when you need some finer control over the
process, it is possible to jump off the fast track and work directly
with an underlying object.  